# Función de _ranking_ 1: Similaridad del coseno

Mas allá del _scoring_ _baseline_ (un tanto _naive_) anterior, la primera función de _ranking_ más elaborada que aplicaremos consistirá en una variante clásica de la similitud coseno (con la norma L1). Se trata esencialmente de construir el vector del documento y el vector de la consulta para luego tomar el producto escalar como resultado.
En realidad, es exactamente lo mismo que hemos hecho ya para el simple conteo de términos anterior (suponiendo que el vector consulta era siempre un vector binario con 1 en los términos contenidos en la consulta, y 0 en el resto). Ahora, sin embargo, en lugar de tomar simplemente los conteos de términos (tanto en la consulta como en el documento), podrían considerarse varias alternativas para calcular el peso (=componente) de cada término, decidiendo:

1. Cómo se calcula exactamente la frecuencia de cada término.
2. Cómo se realiza la ponderación por frecuencia de documento de cada término.
3. La estrategia de normalización seguida.

De nuevo, la figura 6.15 de la página 128 del libro de Manning ([enlace](http://nlp.stanford.edu/IR-book/pdf/06vect.pdf)) nos recuerda todas las posibles alternativas para ello, según la notación SMART.

En lo que sigue discutiremos las opciones para ambos vectores por separado.

## Vectores de consulta

* **Frecuencia del término** (_tf_):
Las frecuencias crudas pueden computarse de forma sencilla a partir de los términos de la consulta. Deberían corresponderse, en la mayoría de los casos, con un simple 1 para cada término que apareciese en la consulta, equivalente a la opción _"boolean"_ (pero no necesariamente, ya que algún término podría aparecer repetido). Dicha frecuencia cruda podría, si se desease, ser escalada sublinealmente (usando el logaritmo). En todo caso, en este notebook mantendremos el enfoque simple del conteo natural de términos, muy similar al vector booleano, dado que la inmensa mayoría de las consultas del _dataset_ no contienen términos repetidos.

* **Frecuencia del documento** (_df_):
Cada uno de los términos en el vector de la consulta deberá entonces ser pesado (=multiplicado) usando el valor de IDF correspondiente para cada término. Usaremos, como ya se comentó antes en este mismo notebook, el IDF computado a partir del corpus de la práctica 1. Recuérdese que, para el caso de palabras que no apareciesen en dicho corpus, se usará la técnica del suavizado Laplaciano (es decir, simplemente sumar 1 en el numerador y en el denominador; esto equivale esencialmente a asumir la existencia de un hipotético documento _"dummy"_ que contiene todos los posibles términos)

* **Normalización** (_norm_):
En ningún caso será necesario normalizar el vector de consulta, ya que cualquier posible normalización se aplicaría por igual al cruzarlo con todos los correspondientes documentos resultado, obteniéndose un simple escalado uniforme de los valores de _scoring_, lo que obviamente no influiría en absoluto en el correspondiente _ranking_ de resultados.

## Vectores de documento

* **Frecuencia del término** (_tf_):
Al igual que con el vector de consulta, podremos utilizar directamente las frecuencias crudas, o, alternativamente, aplicar algún tipo de escalado sublineal. En particular, el escalado sublineal típico es $tf_i = 1 + log(rs_i)$ si $rs_i > 0$, o simplemente $0$ en caso contrario. Así, por ejemplo, para el anterior vector _tf_ del campo **body** del documento _d_, el vector resultante sería $[\text{1+log(10)  1+log(7)  1+log(1)  0}]^T$ (de nuevo, puede encontrarse más información sobre el escalado sublineal del término _tf_ en la página 126, sección 6.4.1 del [libro de Manning](http://nlp.stanford.edu/IR-book/pdf/06vect.pdf)).

* **Frecuencia del documento** (_df_):
No utilizaremos ningún tipo de frecuencia del documento en el vector de documento. En lugar de ello, se incorporará este peso únicamente en el vector de consulta, como se describía en el apartado anterior.

* **Normalización** (_norm_):
En este caso, no podemos usar la normalización del coseno, dado que nuestros archivos de entrenamiento no proporcionan acceso al contenido del documento en sí, sino solo un resumen de campos. Por lo tanto, no sabemos ni qué otros términos aparecen, ni el recuento de los mismos, en el campo **body**. En lugar de ello, por tanto, utilizaremos la normalización de longitud. Además, dado que puede haber enormes discrepancias entre las longitudes de los diferentes campos, dividimos todos los campos por el mismo factor de normalización, la propia longitud del campo **body**. Dado, además, el hecho de que algunos documentos aparecen con una longitud de 0, una buena estrategia es, de nuevo, agregar un valor (p.e. 500), a la longitud del cuerpo de cada documento. El valor concreto a utilizar, además, puede ser también utilizado para experimentar con ésta u otras estrategias de suavizado, y observar su posible influencia en los resultados de _ranking_ obtenidos.

## Pesado relativo de los diferentes campos

Dado un documento $d$ y una consulta $q$, si $qv_q$ es el vector resultante de la consulta y $tf_{d,u}$, $tf_{d,t}$, $tf_{d,b}$, $tf_{d,h}$ y $tf_{d,a}$ son los vectores resultantes para cada uno de los campos **url**, **title**, **body**, **header** and **anchor**, respectivamente, definimos el _scoring_ global neto como (nótese que el símbolo $\cdot$ se usa en la siguiente expresión tanto para el producto escalar entre vectores como para el producto de un escalar por un vector):

$$qv_q \cdot (c_u \cdot tf_{d,u} + c_t \cdot tf_{d,t} + c_b \cdot tf_{d,b} + c_h \cdot tf_{d,h} + c_a \cdot tf_{d,a})$$

Donde $c_u$, $c_t$, $c_b$, $c_h$ y $c_a$ son los pesos dados a los campos **url**, **title**, **body**, **header** y **anchor**, respectivamente.

Por supuesto, para usar la expresión anterior necesitamos determinar de una forma sensata los pesos para cada uno de estos cinco campos. En este sentido, trataremos de escogerlos de forma que la función NDCG de evaluación (que describiremos más adelante) obtenga un valor lo más optimizado posible cuando sea aplicada al conjunto de test completo. Usaremos el conjunto de _training_ para intentar derivar dicho valor óptimo de los cinco parámetros mencionados, para luego evaluar su rendimiento en el conjunto de _test_. En una primera instancia, lo intentaremos hacer manualmente, intentando razonar simplemente sobre la importancia relativa de los diferentes pesos. Al final del notebook sustituiremos esta suerte de "razonamiento manual" por una śencilla técnica de _machine learning_.

Nótese que el valor absoluto de dichos pesos no importará, sólo la relación (ratio) entre ellos (valores relativos), dado que si multiplicamos todos los pesos por la misma constante, el _scoring_ final quedará simplemente multiplicado por dicha constante para todos los documentos por igual, lo que obviamente no afectará en absoluto a la ordenación.

### Esquema de _weighting_ inicial

Proporcionamos aquí un esquema de pesado por defecto de partida, que puede después variarse para intentar mejorar el rendimiento (medido con NDGC). Nótese que:
* En estos primeros pesos por defecto se asigna una importancia del peso de la URL 100 veces superior a la del resto de pesos, a los que, por otro lado, se da un peso equivalente.
* Se añade al conjunto de parámetros ajustables un parámetro de suavizado de la longitud del documento (`smoothing_body_length`), término que podrá ser modificado para influir en la función de _scoring_ final. Dicho término será simplemente sumado a la longitud original de cada documento (con lo que, en todo caso, y como se comentó anteriormente, se evitará siempre la posible división por cero para documentos en los cuales la longitud indicada en el documento del dataset de entrada sea cero). Se deduce, pues, que dar un valor cada vez mayor para este parámetro implicará una influencia progresivamente menor del campo `body_length` original de cada documento particular, ya que el correspondiente factor de influencia de la longitud tenderá con ello a homogeneizarse más para todos los documentos, al disminuir progresivamente el peso relativo del valor inicial de `body_length` en la suma total del denominador.

In [1]:
params_cosine = {
    "url_weight" : 10,
    "title_weight": 0.1,
    "body_hits_weight" : 0.1,
    "header_weight" : 0.1,
    "anchor_weight" : 0.1,
    "smoothing_body_length" : 500,
}

## Clase _CosineSimilarityScorer_

He aquí la definición de una clase para realizar un _scoring_ basado en la similaridad del coseno (basada en la clase `AbstractScorer` definida anteriormente, y reimplementando los métodos adecuados):

In [2]:
# Imports necesarios
import sys
import array
import os
import timeit
import contextlib
import math
import urllib.request
import zipfile
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from collections import OrderedDict, Counter, defaultdict
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Definiciones de clases del notebook anterior (P2a)
class Query:
    """Clase utilizada para almacenar una consulta."""
    def __init__(self, query):
        self.query_words = query.split(" ")

    def __iter__(self):
        for w in self.query_words:
            yield w

    def __eq__(self, other):
        if not isinstance(other, Query):
            return False
        return self.query_words == other.query_words

    def __hash__(self):
        return hash(str(self))

    def __str__(self):
        return " ".join(self.query_words)

    __repr__ = __str__

class Document:
    """Clase utilizada para almacenar información útil para un documento."""
    def __init__(self, url):
        self.url = url        # Cadena
        self.title = None     # Cadena
        self.headers = None   # [Lista de cadenas]
        self.body_hits = None # Diccionario: Término->[Lista de posiciones]
        self.body_length = 0  # Entero
        self.pagerank = 0     # Entero
        self.anchors = None   # Diccionario: Cadena->[Conteo total de ocurrencias (anchor_counts)]

    def __iter__(self):
        for u in self.url:
            yield u

    def __str__(self):
        result = [];
        NEW_LINE = "\n"
        result.append("url: "+ self.url + NEW_LINE);
        if (self.title is not None): result.append("title: " + self.title + NEW_LINE);
        if (self.headers is not None): result.append("headers: " + str(self.headers) + NEW_LINE);
        if (self.body_hits is not None): result.append("body_hits: " + str(self.body_hits) + NEW_LINE);
        if (self.body_length != 0): result.append("body_length: " + str(self.body_length) + NEW_LINE);
        if (self.pagerank != 0): result.append("pagerank: " + str(self.pagerank) + NEW_LINE);
        if (self.anchors is not None): result.append("anchors: " + str(self.anchors) + NEW_LINE);
        return " ".join(result)

    __repr__ = __str__

class Idf:
    """Construye un diccionario para poder devolver el IDF de un término."""
    def __init__(self):
        """Construcción del diccionario IDF"""
        try:
            with open("pa3-data/docs.dict", 'rb') as f:
                docs = pkl.load(f)
            self.total_doc_num = len(docs)
            print("Número total de documentos de la colección:", self.total_doc_num)

            with open("pa3-data/terms.dict", 'rb') as f:
                terms = pkl.load(f)
            self.total_term_num = len(terms)
            print("Número total de términos:", self.total_term_num)

            with open('pa3-data/BSBI.dict', 'rb') as f:
                postings_dict, termsID = pkl.load(f)

            self.idf = {}
            self.raw = {}
            for term_id, term_str in enumerate(terms.id_to_str):
                if term_id in postings_dict:
                    num_docs_with_term = postings_dict[term_id][1]
                    self.raw[term_str] = num_docs_with_term
                    self.idf[term_str] = math.log((self.total_doc_num + 1) / (num_docs_with_term + 1))
        except FileNotFoundError:
            print("¡Ficheros de diccionario de documentos / términos / índice no encontrados!")

    def get_raw(self, term):
        return self.raw.get(term, 0)

    def get_idf(self, term):
        if term in self.idf:
            return self.idf[term]
        else:
            return math.log(self.total_doc_num + 1)

def load_train_data(feature_file_name):
    """Carga los datos de entrenamiento."""
    line = None
    url = None
    anchor_text = None
    query = None
    query_dict = {}
    try:
        with open(feature_file_name, 'r', encoding = 'utf8') as f:
            for line in f:
                token_index = line.index(":")
                key = line[:token_index].strip()
                value = line[token_index + 1:].strip()
                if key == "query":
                    query = Query(value)
                    query_dict[query] = {}
                elif key == "url":
                    url = value;
                    query_dict[query][url] = Document(url);
                elif key == "title":
                    query_dict[query][url].title = str(value);
                elif key == "header":
                    if query_dict[query][url].headers is None:
                        query_dict[query][url].headers = []
                    query_dict[query][url].headers.append(value)
                elif key == "body_hits":
                    if query_dict[query][url].body_hits is None:
                        query_dict[query][url].body_hits = {}
                    temp = value.split(" ",maxsplit=1);
                    term = temp[0].strip();
                    if term not in query_dict[query][url].body_hits:
                        positions_int = []
                        query_dict[query][url].body_hits[term] = positions_int
                    else:
                        positions_int = query_dict[query][url].body_hits[term]
                    positions = temp[1].strip().split(" ")
                    for position in positions:
                        positions_int.append(int(position))
                elif key == "body_length":
                    query_dict[query][url].body_length = int(value);
                elif key == "pagerank":
                    query_dict[query][url].pagerank = int(value);
                elif key == "anchor_text":
                    anchor_text = value
                    if query_dict[query][url].anchors is None:
                        query_dict[query][url].anchors = {}
                elif key == "stanford_anchor_count":
                    query_dict[query][url].anchors[anchor_text] = int(value)
    except FileNotFoundError:
        print(f"Fichero {feature_file_name} no encontrado!")
    return query_dict

class IdMap:
    """Clase auxiliar para almacenar mapeos entre strings e identificadores numéricos de tokens."""
    
    def __init__(self):
        """Constructor de la clase IdMap."""
        self.str_to_id = {}  # Diccionario: string -> id numérico
        self.id_to_str = []  # Lista: índice = id, valor = string
        
    def __len__(self):
        """Devuelve el número de elementos en el mapeo."""
        return len(self.id_to_str)
    
    def _get_str(self, i):
        """Devuelve el string correspondiente al id numérico i.
        
        Args:
            i (int): Identificador numérico.
            
        Returns:
            str: String correspondiente al id.
        """
        return self.id_to_str[i]
    
    def _get_id(self, s):
        """Devuelve el id numérico correspondiente al string s.
        Si el string no existe, lo añade y devuelve su nuevo id.
        
        Args:
            s (str): String del que obtener su id.
            
        Returns:
            int: Identificador numérico del string.
        """
        if s not in self.str_to_id:
            # Si no existe, lo añadimos
            new_id = len(self.id_to_str)
            self.str_to_id[s] = new_id
            self.id_to_str.append(s)
            return new_id
        return self.str_to_id[s]
    
    def __getitem__(self, key):
        """Permite acceso mediante corchetes: idmap[key]
        Si key es int, devuelve el string.
        Si key es str, devuelve el id.
        
        Args:
            key: Puede ser int (devuelve string) o str (devuelve id).
            
        Returns:
            El string o id correspondiente.
        """
        if isinstance(key, int):
            return self._get_str(key)
        elif isinstance(key, str):
            return self._get_id(key)
        else:
            raise TypeError("La clave debe ser int o str")
    
    def __contains__(self, key):
        """Permite usar 'in' para verificar existencia.
        
        Args:
            key: Puede ser int o str.
            
        Returns:
            bool: True si existe, False en caso contrario.
        """
        if isinstance(key, int):
            return 0 <= key < len(self.id_to_str)
        elif isinstance(key, str):
            return key in self.str_to_id
        return False
    
    def __repr__(self):
        """Representación en string del objeto."""
        return f"IdMap(size={len(self)})"
    
class AbstractScorer:
    """ Una clase básica abstracta para un scorer.
        Implementa una funcionalidad básica de construcción de vectores de consulta y de documento.
        Tendrá que ser extendida adecuadamente por cada scorer específico.
    """
    def __init__(self, idf, query_weight_scheme=None, doc_weight_scheme=None):
        self.idf = idf
        self.TFTYPES = ["url", "title", "body_hits", "header", "anchor"]
        # Esquemas por defecto:
        self.default_query_weight_scheme = {"tf": 'n', "df": 'n', "norm": None} # Esquema natural, no, none
        self.default_doc_weight_scheme = {"tf": 'n', "df": 'n', "norm": None}   # Esquema natural, no, none
        self.query_weight_scheme = query_weight_scheme if query_weight_scheme is not None \
                                   else self.default_query_weight_scheme
        self.doc_weight_scheme = doc_weight_scheme if doc_weight_scheme is not None \
                                 else self.default_doc_weight_scheme

    def parse_url(self, url):
        """Parsea la URL del documento, devolviendo un Counter de los tokens encontrados en la URL.
        Args:
            url: el url del que se va a hacer el parsing.
        Returns:
            Lista de tokens del URL (una vez limpios), y Counter resultado.
        """
        if url:
            url_token_in_term = url.replace("http:",".").replace('/','.').replace('?','.') \
                                   .replace('=','.').replace("%20",".").replace("...",".").replace("..",".")\
                                   .replace('-','.').lower();
            url_token = url_token_in_term.strip(".").split('.')
            return url_token, Counter(url_token)
        else:
            return [], Counter([])

    def parse_title(self, title):
        """Parsea el campo title del documento, devolviendo un Counter de los tokens encontrados en el mismo.
        Args:
            title: el title del que se va a hacer el parsing.
        Returns:
            El Counter resultado.
        """
        if title:
            return Counter(title.split(" "))
        else:
            return Counter([])

    def parse_headers(self, headers):
        """Parsea los campos headers del documento, devolviendo un Counter de los tokens encontrados en los mismos.
        Args:
            headers: la lista de headers sobre los que se va a hacer el parsing.
        Returns:
            El Counter resultado.
        """
        headers_token = []
        # BEGIN YOUR CODE
        if headers is not None:
            for header in headers:
                # Dividir cada header en tokens y añadirlos a la lista
                header_tokens = header.split(" ")
                headers_token.extend(header_tokens)
        # END YOUR CODE
        return Counter(headers_token)

    def parse_anchors(self, anchors):
        """Parsea los campos anchors del documento, devolviendo un Counter de los tokens encontrados en los mismos.
        Args:
            anchors: la lista de anchors sobre los que se va a hacer el parsing.
        Returns:
            El Counter resultado.
        """
        anchor_count_map = Counter({})
        if anchors is not None:
            for anchor in anchors:
                count = anchors[anchor]
                anchor_tokens = anchor.split(" ")
                for anchor_token in anchor_tokens:
                    if(anchor_token in anchor_count_map.keys()):
                        anchor_count_map[anchor_token] += count
                    else:
                        anchor_count_map[anchor_token] = count
        return anchor_count_map

    def parse_body_hits(self, body_hits):
        """Parsea los campos body_hits del documento, devolviendo un Counter de los tokens encontrados en los mismos.
        Args:
            body_hits: la lista de anchors sobre los que se va a hacer el parsing.
        Returns:
            El Counter resultado.
        """
        body_hits_count_map = Counter({})
        #BEGIN YOUR CODE
        if body_hits is not None:
            # body_hits es un diccionario: término -> lista de posiciones
            # El conteo es simplemente la longitud de la lista de posiciones
            for term, positions in body_hits.items():
                body_hits_count_map[term] = len(positions)
        #END YOUR CODE
        return body_hits_count_map

    def get_query_vector(self, q, query_weight_scheme = None):
        """ Obtiene un vector numérico para la consulta q.
        Args:
            q (Query): Query("Una consulta determinada")
        Returns:
            query_vec (dict): El vector resultado.
        """
        # En subclases de esta AbstractScorer, podrían tenerse en cuenta todas las
        # posibilidades SMART, usando diferentes esquemas de frecuencia del término (tf),
        # frecuencia de documento (idf) y normalización. En todo caso, nótese que en
        # general no se suele necesitar normalización para la consulta en ningún caso, ya que
        # dicha normalización no variaría con respecto a todos los documentos resultados de una
        # misma consulta, lo que resultaría en un simple factor de escalado común que no
        # afectaría al posterior ranking de los mismos.
        #
        # if query_weight_scheme is None:
        #     query_weight_scheme = self.query_weight_scheme

        query_vec = {}
        ### BEGIN YOUR CODE (FIXME)
        # En nuestro caso base, usaremos simplemente el contador básico de términos, sin
        # normalización ni uso de idf:
        query_vec = Counter(q.query_words)
        ### END YOUR CODE (FIXME)
        return query_vec

    def get_doc_vector(self, q, d, doc_weight_scheme=None):
        """ Obtiene un vector numérico para el documento d.
        Args:
        q (Query) : Query("Una consulta")
        d (Document) : Query("Una consulta")["Un URL"]
        Returns:
        doc_vec (dict) : Un diccionario de conteo de la frecuencia de términos, con un subdiccionario para
                         cada tipo de campo (tipo_de_campo -> (término -> conteo))
                    Ejemplo: "{'url':   {'stanford': 1, 'aoerc': 0, 'pool': 0, 'hours': 0},
                               'title': {'stanford': 1, 'aoerc': 0, 'pool': 0, 'hours': 0},
                               ...
                               }"
        """
        # De nuevo, podrían considerarse todas las posibilidades SMART en las subclases
        # de esta AbstractScorer, si bien en esta clase base nos contentaremos con un simple
        # conteo crudo de los términos en los distintos campos:
        #
        # if doc_weight_scheme is None:
        #    doc_weight_scheme = self.doc_weight_scheme

        doc_vec = {}
        ### BEGIN YOUR CODE (FIXME)
        # Sólo para depurar:
        # print(f"URL:        {d.url}          ->   {self.parse_url(d.url)}")
        # print(f"TITLE:      {d.title}        ->   {self.parse_title(d.title)}")
        # print(f"HEADERS:    {d.headers}      ->   {self.parse_headers(d.headers)}")
        # print(f"ANCHORS:    {d.anchors}      ->   {self.parse_anchors(d.anchors)}")
        # print(f"BODY_HITS:  {d.body_hits}    ->   {self.parse_body_hits(d.body_hits)}")
        #
        # Simple conteo crudo de los términos por campos:
        _, url_counter = self.parse_url(d.url)
        doc_vec['url'] = url_counter
        doc_vec['title'] = self.parse_title(d.title)
        doc_vec['headers'] = self.parse_headers(d.headers)
        doc_vec['anchors'] = self.parse_anchors(d.anchors)
        doc_vec['body_hits'] = self.parse_body_hits(d.body_hits)
        ### END YOUR CODE (FIXME)
        return doc_vec

    # Métodos no implementados en la clase base; en su caso, serán reimplementados en cada scorer concreto:

    def normalize_doc_vec(self, q, d, doc_vec):
        """ Normalizar el vector de documento.
        Args:
            q (Query) : La consulta.
            d (Document) : El documento.
            doc_vec (dict) : El vector de documento
        """
        raise NotImplementedError

    def get_sim_score(self, q, d):
        """ Devuelve la puntuación para una consulta q y documento d dados.
        Args:
            q (Query): la consulta.
            d (Document) : el documento.
        Returns:
            La puntuación para el par (q,d).
        """
        raise NotImplementedError

    def get_net_score(self, q, d):
        """ Calcular el scoring neto entre la consulta y el documento.
        Args:
            q (Query) : La consulta.
            d (Document) : El documento.
        Return:
            score (float) : La puntuación resultado.
        """
        raise NotImplementedError

# Cargar datos necesarios
print("Cargando datos...")
data_dir = 'pa3-data'
theIDF = Idf()
print()
file_name = os.path.join(data_dir, "pa3.signal.train")
query_dict = load_train_data(file_name)
print(f"Datos cargados: {len(query_dict)} consultas")

Cargando datos...
Número total de documentos de la colección: 98998
Número total de términos: 347071

Datos cargados: 731 consultas


In [3]:
class CosineSimilarityScorer(AbstractScorer):

    def __init__(self, idf, query_dict, params, query_weight_scheme=None, doc_weight_scheme=None):
        # Inicializamos clase base "AbstractScorer", ...
        super().__init__(idf, query_weight_scheme=query_weight_scheme, doc_weight_scheme=doc_weight_scheme)
        self.query_dict = query_dict
        # ... y añadimos los parámetros necesarios (5 pesos de los 5 campos + factor suavizado longitud):
        try:
            self.smoothing_body_length = params["smoothing_body_length"]
        except:
            self.smoothing_body_length = 0
        self.weights = {"url": params["url_weight"], "title": params["title_weight"],
                        "headers": params["header_weight"], "anchors": params["anchor_weight"],
                        "body_hits": params["body_hits_weight"]}

    def get_query_vector(self, q):
        """ Usando los vectores de conteo crudos de la clase base, aplica diferentes variantes
            SMART para obtener el correspondiente vector numéricos de consulta modificado.
        Args:
            q (Query): Query("Una consulta determinada")
        Returns:
            query_vec (dict): El vector resultado.
        """
        # Método de conteo de la clase base:
        query_vec = super().get_query_vector(q)

        # Frecuencia de término (implementadas las variantes n y b SMART):
        if self.query_weight_scheme["tf"] == "b":   # Vector query_vec booleano:
            ### BEGIN YOUR CODE (FIXME)
            # Convertir todos los valores > 0 a 1 (representación booleana)
            for term in query_vec:
                if query_vec[term] > 0:
                    query_vec[term] = 1
            ### END YOUR CODE (FIXME)
            
        # Frecuencia de documento (implementadas las variantes n y t SMART):
        if self.query_weight_scheme["df"] == "n":     # No se modifica query_vec:
            pass
        elif self.query_weight_scheme["df"] == "t":   # Modificación IDF de query_vec:
            ### BEGIN YOUR CODE (FIXME)
            # Multiplicar cada término por su IDF
            for term in query_vec:
                query_vec[term] = query_vec[term] * self.idf.get_idf(term)
            ### END YOUR CODE (FIXME)
        return query_vec

    def get_doc_vector(self, q, d):
        """ Usando los vectores de conteo crudos de la clase base, aplica diferentes variantes
            SMART para obtener los correspondientes vectores numéricos de documento modificados.
        Args:
        q (Query) : Query("Una consulta")
        d (Document) : Query("Una consulta")["Un URL"]
        Returns:
        doc_vec (dict) : Vectores numéricos modificados, de nuevo con esquema (tipo_de_campo -> (término -> conteo))
                    Ejemplo: "{'url':   {'stanford': 0.13, 'aoerc': 0, 'pool': 0, 'hours': 0},
                               'title': {'stanford': 0.11, 'aoerc': 0, 'pool': 0, 'hours': 0},
                               ...
                               }"
        """
        # Método de conteo de la clase base:
        doc_vec = super().get_doc_vector(q, d)

        # Frecuencia de término (implementadas las variantes n y l SMART):
        if self.doc_weight_scheme["tf"] == "n":   # No se modifica doc_vec:
            pass
        elif self.doc_weight_scheme["tf"] == "l": # Modificación logarítmica (sublineal) de doc_vec
            ### BEGIN YOUR CODE (FIXME)
            # Para cada campo del documento
            for field_type in doc_vec:
                # Para cada término en ese campo
                for term in doc_vec[field_type]:
                    raw_count = doc_vec[field_type][term]
                    if raw_count > 0:
                        # Aplicar escalado logarítmico: 1 + log(raw_count)
                        doc_vec[field_type][term] = 1 + math.log(raw_count)
                    # Si raw_count == 0, se mantiene en 0
            ### END YOUR CODE (FIXME)
            
        # Normalización:
        if self.doc_weight_scheme['norm'] == "default":
            doc_vec = self.normalize_doc_vec(q, d, doc_vec)

        return doc_vec

    def get_sim_score(self, q, d, field_type):
        """ Cálculo del scoring para un campo individual:
        Args:
            q (Query) : La consulta.
            d (Document) : El documento.
            field_type (str) : El campo del que se usará el vector de documento.
        Return:
            score (float) : El scoring individual (para el campo field_type) del par (q,d):
        """
        ### BEGIN YOUR CODE (FIXME)
        score = 0
        
        # Obtener vectores de consulta y documento
        query_vec = self.get_query_vector(q)
        doc_vec = self.get_doc_vector(q, d)
        
        # Obtener el vector del campo específico
        field_vec = doc_vec[field_type]
        
        # Calcular producto escalar: sumar query_vec[term] * field_vec[term] para cada término
        for term in query_vec:
            if term in field_vec:
                score += query_vec[term] * field_vec[term]
        
        ### END YOUR CODE (FIXME)
        return score

    def get_net_score(self, q, d):
        """ Cálculo del scoring global (neto), usando los cinco pesos:
        Args:
            q (Query) : La consulta.
            d (Document) : El documento.
        Return:
            score (float) : El scoring global (neto, sumando todos los campos) del par (q,d):
        """
        ### BEGIN YOUR CODE (FIXME)
        score = 0
        
        # Para cada campo, calcular su scoring individual y multiplicarlo por su peso
        for field_type in ["url", "title", "headers", "anchors", "body_hits"]:
            field_score = self.get_sim_score(q, d, field_type)
            score += self.weights[field_type] * field_score
        
        ### END YOUR CODE (FIXME)
        return score

    ## Normalización
    def normalize_doc_vec(self, q, d, doc_vec):
        """ Normalización del vector de documento:
        Damos una normalización uniforme basada en la longitud del documento, tal
        y como se discutió en el item "Normalización" del anterior apartado.
        Es decir, dividimos cada componente del vector de documento por
        (longitud_del_cuerpo_del_documento + factor_de_suavizado).
        
        Args:
            q (Query) : La consulta.
            d (Document) : El documento.
            doc_vec (dict) : El vector de documento.
        Return:
            doc_vec (dict) : El vector de documento tras la normalización.
        """
        ### BEGIN YOUR CODE (FIXME)
        # Calcular el factor de normalización
        norm_factor = d.body_length + self.smoothing_body_length
        
        # Normalizar cada campo dividiendo por el factor de normalización
        for field_type in doc_vec:
            for term in doc_vec[field_type]:
                doc_vec[field_type][term] = doc_vec[field_type][term] / norm_factor
        
        ### END YOUR CODE (FIXME)
        # print(d.body_length, self.smoothing_body_length)

        return doc_vec

He aquí una primera prueba sencilla de _scoring_ de un par ($q$,$d$) usando esta similaridad del coseno, en particular usando el esquema SMART _ddd.qqq_ = _nnn.bnn_:

In [4]:
q = Query("stanford aoerc pool hours")
d = query_dict[q]['http://events.stanford.edu/2014/February/18/']
doc_weight_scheme = {"tf": 'n', "df": 'n', "norm": None}
query_weight_scheme = {"tf": 'b', "df": 'n', "norm": None}
cs = CosineSimilarityScorer(theIDF, query_dict, params_cosine, query_weight_scheme, doc_weight_scheme)
print('Vector consulta: ', cs.get_query_vector(q), '\n')
print('Vector de documento original:\n', cs.get_doc_vector(q, d), '\n')
print("---")
print("Scoring campo url:", cs.get_sim_score(q,d,"url"))
print("Scoring campo title:", cs.get_sim_score(q,d,"title"))
print("Scoring campo headers:", cs.get_sim_score(q,d,"headers"))
print("Scoring campo anchors:", cs.get_sim_score(q,d,"anchors"))
print("Scoring campo body_hits:", cs.get_sim_score(q,d,"body_hits"))
print("\nScoring neto:", cs.get_net_score(q,d))

Vector consulta:  Counter({'stanford': 1, 'aoerc': 1, 'pool': 1, 'hours': 1}) 

Vector de documento original:
 {'url': Counter({'events': 1, 'stanford': 1, 'edu': 1, '2014': 1, 'february': 1, '18': 1}), 'title': Counter({'events': 1, 'at': 1, 'stanford': 1, 'tuesday': 1, 'february': 1, '18': 1, '2014': 1}), 'headers': Counter({'stanford': 5, 'university': 1, 'event': 1, 'calendar': 1, 'teaching': 1, 'sex': 1, 'at': 1, 'rodin': 1, 'the': 1, 'complete': 1, 'collection': 1, 'rec': 1, 'trx': 1, 'suspension': 1, 'training': 1, 'memorial': 1, 'church': 1, 'open': 1, 'visiting': 1, 'hours': 1, 'alternative': 1, 'transportation': 1, 'counseling': 1, 'tm': 1, '3': 1, 'hour': 1, 'univ': 1, 'shc': 1, 'employees': 1, 'retirees': 1, 'family': 1, 'members': 1}), 'anchors': Counter(), 'body_hits': Counter({'stanford': 10, 'aoerc': 7, 'pool': 1})} 

---
Scoring campo url: 1
Scoring campo title: 1
Scoring campo headers: 6
Scoring campo anchors: 0
Scoring campo body_hits: 18

Scoring neto: 12.5


La salida tendría que ser la siguiente:

Y he aquí, para el mismo par ($q$,$d$), algunos posibles vectores alternativos, obtenidos usando diferentes variantes SMART, tanto para la consulta $q$ como para el documento $d$:

In [5]:
q = Query("stanford aoerc pool hours")
d = query_dict[q]['http://events.stanford.edu/2014/February/18/']

query_weight_scheme, doc_weight_scheme = None, None

query_weight_scheme = {"tf": 'b', "df": 'n', "norm": None}
cs = CosineSimilarityScorer(theIDF, query_dict, params_cosine, query_weight_scheme, doc_weight_scheme)
print('Vector consulta original: ', cs.get_query_vector(q), '\n')

query_weight_scheme = {"tf": 'b', "df": 't', "norm": None}
cs = CosineSimilarityScorer(theIDF, query_dict, params_cosine, query_weight_scheme, doc_weight_scheme)
print('Vector consulta IDF: ', cs.get_query_vector(q), '\n')

doc_weight_scheme = {"tf": 'n', "df": 'n', "norm": None}
cs = CosineSimilarityScorer(theIDF, query_dict, params_cosine, query_weight_scheme, doc_weight_scheme)
print('Vector de documento original:\n', cs.get_doc_vector(q, d), '\n')
print("-----")

doc_weight_scheme = {"tf": 'n', "df": 'n', "norm": "default"}
cs = CosineSimilarityScorer(theIDF, query_dict, params_cosine, query_weight_scheme, doc_weight_scheme)
print('Vector de documento normalizado:\n', cs.get_doc_vector(q, d), '\n')
print("-----")

doc_weight_scheme = {"tf": 'l', "df": 'n', "norm": None}
cs = CosineSimilarityScorer(theIDF, query_dict, params_cosine, query_weight_scheme, doc_weight_scheme)
print('Vector de documento con escalado logarítmico de tf:\n', cs.get_doc_vector(q, d), '\n')
print("-----")

doc_weight_scheme = {"tf": 'l', "df": 'n', "norm": "default"}
cs = CosineSimilarityScorer(theIDF, query_dict, params_cosine, query_weight_scheme, doc_weight_scheme)
print('Vector de documento con escalado logarítmico y normalizado:\n', cs.get_doc_vector(q, d), '\n')
print("-----")

Vector consulta original:  Counter({'stanford': 1, 'aoerc': 1, 'pool': 1, 'hours': 1}) 

Vector consulta IDF:  Counter({'aoerc': 11.502865028055611, 'pool': 5.633568114921836, 'hours': 2.9662614431195498, 'stanford': 0.3295747967117297}) 

Vector de documento original:
 {'url': Counter({'events': 1, 'stanford': 1, 'edu': 1, '2014': 1, 'february': 1, '18': 1}), 'title': Counter({'events': 1, 'at': 1, 'stanford': 1, 'tuesday': 1, 'february': 1, '18': 1, '2014': 1}), 'headers': Counter({'stanford': 5, 'university': 1, 'event': 1, 'calendar': 1, 'teaching': 1, 'sex': 1, 'at': 1, 'rodin': 1, 'the': 1, 'complete': 1, 'collection': 1, 'rec': 1, 'trx': 1, 'suspension': 1, 'training': 1, 'memorial': 1, 'church': 1, 'open': 1, 'visiting': 1, 'hours': 1, 'alternative': 1, 'transportation': 1, 'counseling': 1, 'tm': 1, '3': 1, 'hour': 1, 'univ': 1, 'shc': 1, 'employees': 1, 'retirees': 1, 'family': 1, 'members': 1}), 'anchors': Counter(), 'body_hits': Counter({'stanford': 10, 'aoerc': 7, 'pool': 1

La salida debería ser la siguiente: